In [ ]:
import wave
from pyaudio import PyAudio,paInt16
import os
from aip import AipSpeech
import ffmpy #视频音频转码模块
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import sys
import pygame
from pydub import AudioSegment
import requests

os.chdir('E:\我的东西\新建文件夹\文件\课件\信工课件\电子系统综合设计\半成品\代码文件')
framerate=8000
NUM_SAMPLES=2000
channels=1
sampwidth=2
TIME=1
filename_record = 'audio_record.wav'
filename_answer = 'audio.wav'
APP_ID = '11169887'
API_KEY = 'TQypLIsDnr4XwzfyKGLqMsfD'
SECRET_KEY = 'bc5efee36b796c2b467dba12c2a080b0'


def save_wave_file(filename,data):
    '''save the date to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b"".join(data))
    wf.close()

def my_record():
    pa=PyAudio()
    stream=pa.open(format = paInt16,channels=1,
                   rate=framerate,input=True,
                   frames_per_buffer=NUM_SAMPLES)
    my_buf=[]
    count=0
    print('Recording:',end='')
    while count<TIME*20:#控制录音时间
        string_audio_data = stream.read(NUM_SAMPLES)
        my_buf.append(string_audio_data)
        count+=1
        print('.',end='')
    print()
    stream.close()
    return my_buf

chunk=2048 #好像是分段记录时每次记录的点数
def play(filename):
    chunk = 1024
    filename_tmp = 'tmp.wav'
    try:
        sound = AudioSegment.from_file(filename, format=str.split(filename,'.')[1])
    except:
        filename_answer = 'tmp.wav'
        if os.path.exists(filename_answer):
            os.remove(filename_answer)
        ff = ffmpy.FFmpeg( 
        inputs={filename: None},
        outputs={filename_answer: None}
        )
        ff.run() #美中不足的时没有学会怎么执行覆盖文件操作，文件存在时写入操作会被直接跳过，故现在只能暂时在转码播放后直接将其移除            
        sound = AudioSegment.from_file(filename_answer, format=str.split(filename_answer,'.')[1])        
    p=PyAudio()
    stream=p.open(format=p.get_format_from_width(sound.sample_width),channels= \
        sound.channels,rate=sound.frame_rate,output=True)
    i=0
    while True:
        data = sound.raw_data[i*chunk : (i+1)*chunk]
        if data==b'':break
        stream.write(data) #这一行是播放语音
        i = i + 1
    stream.close()
    p.terminate()       
    

# 读取文件
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read() 
    
def get_weather(message):
    date_heard = re.search(u'.*今天'.encode('utf8'), message)
    if date_heard is not None:
        day = 0
    else:
        date_heard = re.search(u'.*明天'.encode('utf8'), message)
        if date_heard is not None:
            day = 1
        else: 
            date_heard = re.search(u'.*后天'.encode('utf8'), message)
            if date_heard is not None:
                day = 2
    resp=urlopen('http://www.weather.com.cn/weather/101280101.shtml')
    soup=BeautifulSoup(resp,'html.parser')
    content = soup.findAll('li', class_="sky") #返回一个链表，每个元素是一个标签包括一天的天气内容
    tagToday=content[day].find('p', class_="tem")
    try:
        temperatureHigh=tagToday.span.string
    except AttributeError as e:
        temperatureHigh=tagToday.find_next('p', class_="tem").span.string
    temperatureLow=tagToday.i.string
    weather=soup.find('p', class_="wea").string
    tagWind=soup.find('p',class_="win")
    winL=tagWind.i.string
    time_now = datetime.datetime.now()
    delta = datetime.timedelta(days=day)
    time_day = time_now + delta
    date_string = str(time_day.month) + '月' + str(time_day.day) + '日' 
    name_date = ['今', '明', '后']
    result_weather = name_date[day] + '天是：'+ date_string + ',风级：' + winL + ',最低温度：' + temperatureLow.replace('℃','度') + ',最高温度：' + temperatureHigh + '度' + ',天气：'+ weather
    return result_weather

def say_answer(message_answer): #client需要在外部创建一个公用的，因为每次向百度语音API申请一个太花费时间了
    result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 3}) 
    if not isinstance(result_tts, dict):
        with open('audio_raw.wav', 'wb') as f:
            f.write(result_tts)
    play('audio_raw.wav')    

def listen_music(keyword):
# keyword = input("请输入想要听的歌曲：")
    url = "http://songsearch.kugou.com/song_search_v2?callback=jQuery1124006980366032059648_1518578518932&keyword="+keyword+"&page=1&pagesize=30&userid=-1&clientver=&platform=WebFilter&tag=em&filter=2&iscorrection=1&privilege_filter=0&_=1518578518934"
    content = requests.get(url)
    if re.findall('"FileHash":"(.*?)"',content.text) == []:
        message_answer = '没有搜索到对应的歌曲'
        result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 0}) 
        with open('audio_answer.wav', 'wb') as f:
            f.write(result_tts)
        play('audio_answer.wav')
    else:
        filehash = re.findall('"FileHash":"(.*?)"',content.text)[0]
        songname = re.findall('"SongName":"(.*?)"',content.text)[0].replace("<\\/em>","").replace("<em>","") #即将播放的歌曲名
        hash_url = "http://www.kugou.com/yy/index.php?r=play/getdata&hash="+filehash
        hash_content = requests.get(hash_url)
        play_url = re.findall('"play_url":"(.*?)"',hash_content.text)
        play_url = ' '.join(play_url)
        real_download_url = play_url.replace("\\","")
        print("客官，请稍等一下，好音乐马上呈上！")
        # with open(songname+".mp3","wb")as fp:
        with open("music.mp3","wb")as fp:
            fp.write(requests.get(real_download_url).content)
        play('music.mp3')
        print('下载完成，敬请收听！')
    
    
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY) 
filename_record = 'AudioBase_DIY/ask_music1.wav'
# data = my_record()
# save_wave_file(filename_record,data)
# message = result_weather.encode('utf8')
# wf=wave.open(filename_record,'rb')
# rate=wf.getframerate()    #照目前的情况是8000，如果有变化，再用上这个语句
result_stt = client.asr(get_file_content(filename_record), 'wav', 8000, {'dev_pid': '1537'}) #dev_pid=1537是纯中文普通话，1536支持简单英文
#对wav文件无法识别没有RIFF头的文件，不能直接传data
# wf.close()

if result_stt['err_msg'] != 'success.':
    print('没有识别到内容')
else:
    print(result_stt['result'])
    message = str(result_stt['result']).encode('utf8')
#     message = '魔镜魔镜今天天气怎么样'.encode('utf8')
    search_result = re.search(u'.*魔镜'.encode('utf8'), message)
    if search_result is not None: #接收到了唤醒语，开始工作
        flag = 1
        search_result = re.search(u'.*天气'.encode('utf8'), message)
        if search_result is not None:
            message_answer = get_weather(message)
            say_answer(message_answer)
        else: 
            pattern = re.compile(u'歌|音乐'.encode('utf8'))
            search_result = re.findall(pattern, message)
            if len(search_result) != 0: #search失败返回的是Nonetype，findall失败返回一个空list
                message_answer = '你想听什么' + search_result[0].decode('utf8')
                say_answer(message_answer)
                # data = my_record() #再接收一波语音
                # save_wave_file(filename_record,data)
                filename_record = 'AudioBase_DIY/music_name3.wav' #临时语句，仅供测试
                result_stt = client.asr(get_file_content(filename_record), 'wav', 8000, {'dev_pid': '1537'})
                keyword = result_stt['result'][0].replace('，','') #str类型即可
                print(keyword)
#                 keyword = '告白气球'
                listen_music(keyword)                
            else:
                flag = 0
    #         message_answer = '你在说什么我听不见'
#         if flag:
#             result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 3}) 
#             if not isinstance(result_tts, dict):
#                 with open('audio_raw.wav', 'wb') as f:
#                     f.write(result_tts)
#             play('audio_raw.wav')

    # 首先实现：读取A23_0.wav，能将识别内容显示在屏幕上（已完成）
    # 然后实现：将显示在屏幕上的识别内容转成音频
    # 接着给音频加上RIFF头
    # 最后播放音频